## Import necessary libraries

In [34]:
%pip install langchain[llm]
%pip install gpt4all

langchain[llm] is already installed.
gpt4all is already installed.


In [35]:
from gpt4all import GPT4All
import os
import requests
from tqdm.notebook import tqdm
from pathlib import Path
import platform

## Configuration Constants

In [36]:
# Define the URL, directory, and filename for your model
MODEL_URL = "https://huggingface.co/nomic-ai/gpt4all-falcon-ggml/blob/main/ggml-model-gpt4all-falcon-q4_0.bin"
MODEL_DIR = "models"
MODEL_FILENAME = "ggml-model-gpt4all-falcon-q4_0.bin"
MODEL_PATH = None  # Define MODEL_PATH with a default value

# Check if the code is running in Google Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

# Set MODEL_PATH based on the environment
if IN_COLAB:
    # Code is running in Google Colab
    MODEL_PATH = Path(MODEL_DIR) / MODEL_FILENAME
else:
    # Determine the current operating system
    current_os = platform.system()
    if current_os == "Windows":
        # Convert the path to a string with forward slashes and add '..' before MODEL_DIR
        MODEL_PATH = (Path("..") / MODEL_DIR / MODEL_FILENAME).as_posix()
    else:
        # Convert the path to a string with forward slashes and add '..' before MODEL_DIR
        MODEL_PATH = (Path("..") / MODEL_DIR / MODEL_FILENAME).as_posix()

# Check if MODEL_PATH is defined
if MODEL_PATH is not None:
    print(f"MODEL_PATH: {MODEL_PATH}")
    # You can use MODEL_PATH to access the path to your model file.
else:
    print("MODEL_PATH is not defined. Please check your environment.")


MODEL_PATH: ../models/ggml-model-gpt4all-falcon-q4_0.bin


In [37]:
# Heading: Download Model
print("Download Model")

def download_model():
    """
    Download the model file if it doesn't exist locally.
    """
    # Check if the model file already exists
    if MODEL_PATH and not os.path.exists(MODEL_PATH):
        try:
            # Ensure the directory exists
            os.makedirs(MODEL_DIR, exist_ok=True)

            print(f"Downloading model from {MODEL_URL}...")
            response = requests.get(MODEL_URL, stream=True)  # Use streaming to download in chunks
            total_size = int(response.headers.get('content-length', 0))
            block_size = 1024  # 1 KB

            # Create a progress bar using tqdm
            with open(MODEL_PATH, 'wb') as file, tqdm(
                total=total_size, unit='B', unit_scale=True, unit_divisor=1024
            ) as pbar:
                for data in response.iter_content(block_size):
                    file.write(data)
                    pbar.update(len(data))

            print(f"Downloaded model to {MODEL_PATH}")
        except Exception as e:
            print(f"An error occurred: {str(e)}")
    else:
        print("Model file already exists. Skipping download.")

# Call the download_model function to download the model if necessary
download_model()

Download Model
Model file already exists. Skipping download.


## Importing Models in Langchain

### GPT4All models

In [38]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import GPT4All
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

### Asking Question

In [39]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [40]:
# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

# Verbose is required to pass to the callback manager
llm = GPT4All(model=str(MODEL_PATH), callbacks=callbacks, verbose=True)

Found model file at  ../models/ggml-model-gpt4all-falcon-q4_0.bin


KeyboardInterrupt: 

In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
output = llm_chain("How to make milk tea?")



1. Boil water in a pot.
2. Add loose leaf tea (e.g. green tea, jasmine tea) into the pot.
3. Let it steep for 5-10 minutes.
4. Strain the tea into another pot.
5. Add milk and sweetener (e.g. sugar, honey) to taste.
6. Heat the mixture on low heat until it's warm.
7. Pour the tea into a cup and enjoy!

In [ ]:
output.keys()

dict_keys(['question', 'text'])

## User Defined Question

In [ ]:
input = input("Ask me anything:")
output = llm_chain(str(input))
answer = output['text']
print(answer)

TypeError: 'str' object is not callable

In [ ]:
print(f"You asked: {output['question']}")

You asked: what are major machine learning algorithms
